<a href="https://colab.research.google.com/github/decay00/programming-for-data-an/blob/main/laba_3/laba_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [3]:
import pandas as pd

exoplanet_catolog = pd.read_csv('exoplanet_catalog.csv', usecols=['# name', 'mass', 'radius', 'discovered'], dtype={'# name': str, 'mass': float, 'radius': float, 'discovered': float})

exoplanet = pd.DataFrame(data=exoplanet_catolog)
exoplanet = exoplanet.rename(columns={'# name': 'name'})

exoplanet.dropna(inplace=True)

print(exoplanet.isna().sum())
exoplanet.duplicated().sum()
exoplanet = exoplanet.reset_index(drop=True)
print(exoplanet.info())

name          0
mass          0
radius        0
discovered    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1055 non-null   object 
 1   mass        1055 non-null   float64
 2   radius      1055 non-null   float64
 3   discovered  1055 non-null   float64
dtypes: float64(3), object(1)
memory usage: 33.1+ KB
None


# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [4]:
print(exoplanet.head(30))

                name      mass  radius  discovered
0        1RXS 1609 b  14.00000  1.7000      2008.0
1       2M 0122-24 b  20.00000  1.0000      2013.0
2       2M 0219-39 b  13.90000  1.4400      2015.0
3       2M 2140+16 b  20.00000  0.9200      2010.0
4       2M 2206-20 b  30.00000  1.3000      2010.0
5          2M1510A a  40.00000  1.5700      2020.0
6          2M1510A b  39.30000  1.5700      2020.0
7   2MASS J0348-6022  43.00000  0.9100      2021.0
8   2MASS J0407+1546  67.00000  0.9700      2021.0
9   2MASS J1219+3128  49.00000  0.9700      2021.0
10          51 Eri b   2.60000  1.1100      2015.0
11          51 Peg b   0.47000  1.9000      1995.0
12          55 Cnc e   0.02703  0.1737      2004.0
13         AD 3116 b  54.20000  1.0200      2017.0
14          AU Mic b   0.05300  0.3908      2020.0
15          AU Mic c   0.04630  0.3131      2020.0
16       BD+20 594 b   0.07000  0.2300      2016.0
17      COCONUTS-2 b   6.40000  1.1200      2021.0
18          CT Cha b  17.00000 

**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.

![image.png](attachment:image.png)

В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [5]:
exoplanet.groupby('discovered').count()

,name,mass,radius
discovered,,,
1995.0,1,1,1
1996.0,1,1,1
1999.0,1,1,1
2000.0,2,2,2
2001.0,1,1,1
2002.0,1,1,1
2004.0,7,7,7
2005.0,4,4,4
2006.0,10,10,10


Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [6]:
exo_number = exoplanet.groupby('discovered')['radius'].count()
print(exo_number)

discovered
1995.0      1
1996.0      1
1999.0      1
2000.0      2
2001.0      1
2002.0      1
2004.0      7
2005.0      4
2006.0     10
2007.0     19
2008.0     25
2009.0     15
2010.0     57
2011.0     96
2012.0     73
2013.0     96
2014.0    105
2015.0     57
2016.0    106
2017.0     63
2018.0    114
2019.0     71
2020.0     88
2021.0     42
Name: radius, dtype: int64


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [7]:
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()
print(exo_radius_sum)

discovered
1995.0     1.900000
1996.0     1.060000
1999.0     1.380000
2000.0     2.007000
2001.0     0.921000
2002.0     1.200000
2004.0     6.789700
2005.0     4.789000
2006.0    20.355000
2007.0    24.334600
2008.0    34.329000
2009.0    15.366794
2010.0    56.828660
2011.0    77.967374
2012.0    50.074507
2013.0    69.372100
2014.0    55.268000
2015.0    52.684740
2016.0    97.986000
2017.0    51.597620
2018.0    90.195760
2019.0    62.114410
2020.0    65.119180
2021.0    31.915760
Name: radius, dtype: float64


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [8]:
exo_radius_mean = exo_radius_sum/exo_number
print(exo_radius_mean)

discovered
1995.0    1.900000
1996.0    1.060000
1999.0    1.380000
2000.0    1.003500
2001.0    0.921000
2002.0    1.200000
2004.0    0.969957
2005.0    1.197250
2006.0    2.035500
2007.0    1.280768
2008.0    1.373160
2009.0    1.024453
2010.0    0.996994
2011.0    0.812160
2012.0    0.685952
2013.0    0.722626
2014.0    0.526362
2015.0    0.924294
2016.0    0.924396
2017.0    0.819010
2018.0    0.791191
2019.0    0.874851
2020.0    0.739991
2021.0    0.759899
Name: radius, dtype: float64


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [9]:
def user_genres(group):
    for name, value in group:
        if len(value) > 50:
            return name

#Задача 1
yandex_music = pd.read_csv('yandex_music.csv')

new_names = ['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'day']
yandex_music = yandex_music.set_axis(new_names, axis='columns')
yandex_music.dropna(subset=['genre_name'], inplace=True)

genre_grouping = yandex_music.groupby('user_id')
genre_counting = genre_grouping['genre_name'].count()

print(genre_counting.head(30))

serach_id = user_genres(genre_grouping)
print(serach_id)




user_id
10004220    3
100173ED    1
1001CE90    7
1002410A    3
100261F9    2
1004AAE6    1
1004C5D8    1
1005BECA    1
10089377    3
100AB021    1
100ACFBC    1
10101201    2
10107FC7    2
10112F4C    2
1016144E    1
1016C39F    2
10180175    1
101861D1    2
10189D10    8
101AAA19    1
101B046A    2
101B53F1    1
101BBC61    1
101C12A4    2
101D714C    1
101E910F    1
10211433    1
10222F55    1
1022B77E    1
10246DCB    2
Name: genre_name, dtype: int64
7D166C63


# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.

![image.png](attachment:image.png)


Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [10]:
print(exoplanet.sort_values(by='radius').head(30))

             name      mass   radius  discovered
559   Kepler-37 b  0.010000  0.02600      2013.0
432  Kepler-102 b  0.013530  0.04200      2014.0
470  Kepler-138 b  0.000210  0.04700      2014.0
654   Kepler-62 c  0.013000  0.04800      2013.0
589   Kepler-42 d  0.003000  0.05100      2011.0
433  Kepler-102 c  0.009440  0.05200      2014.0
345      K2-266 c  0.000910  0.06300      2018.0
588   Kepler-42 c  0.006000  0.06500      2011.0
560   Kepler-37 c  0.037760  0.06700      2013.0
851  TRAPPIST-1 h  0.001041  0.06740      2017.0
422      KOI-55 b  0.001400  0.06800      2011.0
847  TRAPPIST-1 d  0.001300  0.06890      2016.0
587   Kepler-42 b  0.009000  0.07000      2011.0
572  Kepler-408 b  0.015730  0.07300      2014.0
441  Kepler-106 b  0.016680  0.07300      2014.0
469  Kepler-131 c  0.026000  0.07500      2014.0
570  Kepler-406 c  0.009000  0.07600      2014.0
500   Kepler-20 e  0.009700  0.07700      2011.0
447  Kepler-107 d  0.011960  0.07700      2014.0
423      KOI-55 c  0

Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [11]:
print(exoplanet[exoplanet['radius'] < 1])

                  name      mass   radius  discovered
3         2M 2140+16 b  20.00000  0.92000      2010.0
7     2MASS J0348-6022  43.00000  0.91000      2021.0
8     2MASS J0407+1546  67.00000  0.97000      2021.0
9     2MASS J1219+3128  49.00000  0.97000      2021.0
12            55 Cnc e   0.02703  0.17370      2004.0
...                ...       ...      ...         ...
1036  WISE 1217+16 A b  22.00000  0.96000      2012.0
1041        Wolf 940 b  26.00000  0.92000      2009.0
1043           XO-2N b   0.62000  0.97300      2007.0
1049  ZTF J0038+2030 b  59.30000  0.78300      2021.0
1053          pi Men c   0.01517  0.16719      2018.0

[565 rows x 4 columns]


Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [12]:
print(exoplanet[exoplanet['discovered'] == 2014])

             name    mass  radius  discovered
86       GU Psc b  11.000   1.350      2014.0
130    HAT-P-49 b   1.730   1.413      2014.0
136    HAT-P-54 b   0.760   0.944      2014.0
163     HATS-15 b   2.170   1.105      2014.0
190      HATS-4 b   1.323   1.020      2014.0
...           ...     ...     ...         ...
1010    WASP-74 b   0.826   1.248      2014.0
1020    WASP-83 b   0.300   1.040      2014.0
1023  WASP-87 A b   2.210   1.385      2014.0
1025    WASP-89 b   5.900   1.040      2014.0
1030  WASP-94 A b   0.452   1.720      2014.0

[105 rows x 4 columns]


А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [13]:
exo_small_14 = exoplanet[(exoplanet['radius'] < 1) & (exoplanet['discovered'] == 2014)]

print(exo_small_14)

             name    mass  radius  discovered
136    HAT-P-54 b  0.7600   0.944      2014.0
201      HATS-5 b  0.2370   0.912      2014.0
212      HATS-6 b  0.3190   0.998      2014.0
269  HIP 116454 b  0.0372   0.226      2014.0
412    KOI-1257 b  1.4500   0.940      2014.0
..            ...     ...     ...         ...
698   Kepler-95 b  0.0410   0.305      2014.0
699   Kepler-96 b  0.0270   0.238      2014.0
700   Kepler-97 b  0.0110   0.132      2014.0
701   Kepler-98 b  0.0110   0.178      2014.0
702   Kepler-99 b  0.0190   0.132      2014.0

[78 rows x 4 columns]


Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [14]:
print(exo_small_14.sort_values(by='radius', ascending=False))

             name     mass  radius  discovered
212      HATS-6 b  0.31900   0.998      2014.0
593  Kepler-425 b  0.25000   0.978      2014.0
586  Kepler-419 b  2.50000   0.960      2014.0
136    HAT-P-54 b  0.76000   0.944      2014.0
412    KOI-1257 b  1.45000   0.940      2014.0
..            ...      ...     ...         ...
572  Kepler-408 b  0.01573   0.073      2014.0
441  Kepler-106 b  0.01668   0.073      2014.0
433  Kepler-102 c  0.00944   0.052      2014.0
470  Kepler-138 b  0.00021   0.047      2014.0
432  Kepler-102 b  0.01353   0.042      2014.0

[78 rows x 4 columns]


Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [15]:
music_user = yandex_music[(yandex_music['user_id'] == serach_id) & (yandex_music['time'] != 0)]

sum_music_user = music_user.groupby('genre_name')['time'].sum()
print(sum_music_user.head())

count_music_user = music_user.groupby('genre_name')['genre_name'].count()
print(count_music_user.head(15))
print(f'count_music_user = {count_music_user.shape[0]}')

final_sum = sum_music_user.sort_values(ascending=False)
print(final_sum)

final_count = count_music_user.sort_values(ascending=False)
print(final_count)


genre_name
classical            20:26:32
country              08:39:24
dance        14:11:5614:23:32
folk                 13:52:18
folkrock             14:29:08
Name: time, dtype: object
genre_name
classical      1
country        1
dance          2
folk           1
folkrock       1
ghazal         1
hip            1
jazz          16
pop           15
punk           1
reggae         2
rnb            1
rock           5
schlager       5
soundtrack     1
Name: genre_name, dtype: int64
count_music_user = 15
genre_name
pop           20:34:4408:46:5114:23:3008:25:3314:22:4309:04:...
hip                                                    20:29:56
jazz          20:29:5314:23:2714:16:1014:23:0608:39:3914:09:...
classical                                              20:26:32
folkrock                                               14:29:08
rock                   14:12:0214:23:4308:35:5808:56:3008:35:58
dance                                          14:11:5614:23:32
soundtrack                         